# Testing the Weaviate Interface Subpackage

This notebook serves as both a testing suite and comprehensive documentation for the `weaviate_interface` subpackage. It demonstrates various functionalities provided by the subpackage, including connection management, schema operations, CRUD operations, searching, batch operations, and aggregations using the `ProductService` and `RouteService`.

## 1. Setup and Initialization

First, we import the necessary modules and initialize the `WeaviateInterface`.

In [1]:
from config import Config
from weaviate_interface import WeaviateInterface
import logging

logging.basicConfig(level=logging.INFO)

# Load configuration
config = Config()

# Initialize WeaviateInterface
weaviate_interface = WeaviateInterface(
    url=config.WEAVIATE_URL,
    openai_key=config.OPENAI_API_KEY,
)

# Access the client and schema manager
client = weaviate_interface.client
schema_manager = weaviate_interface.schema

## 2. Connection Testing

We test the connection to ensure the Weaviate instance is ready for operations.

In [2]:
async def test_connection():
    print("Testing connection to Weaviate...")
    await client.connect()
    is_ready = await client.is_ready()
    print(f"Weaviate is ready: {is_ready}")
    await client.close()


await test_connection()

Testing connection to Weaviate...


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"


Weaviate is ready: True


## 3. Schema Validation and Reset

We validate the current schema against the expected schema defined in `schema.py`. If the schema is not valid, we reset it.

In [3]:
async def test_schema():
    print("\nValidating schema...")
    await client.connect()
    is_valid = await schema_manager.is_valid()
    print(f"Is schema valid? {is_valid}")

    if not is_valid:
        print("Resetting schema...")
        await schema_manager.reset_schema()
        print("Schema reset complete.")
        is_valid = await schema_manager.is_valid()
        print(f"Is schema valid after reset? {is_valid}")

    schema_info = await schema_manager.info()
    print("\nSchema Info:")
    print(schema_info)
    await client.close()


await test_schema()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Validating schema...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:weaviate_interface.schema_manager:Class mismatch: existing {'Route', 'Product'}, expected {'ProductSearchResult', 'ProductDataChunk', 'Route', 'Product', 'RawProductData'}
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/Product "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/Route "HTTP/1.1 200 OK"


Is schema valid? False
Resetting schema...


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:weaviate_interface.schema_manager:Schema reset successfully
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:weaviate_interface.schema_manager:Mismatch in property product_id
INFO:weaviate_interface.schema_manager:Property mismatch for RawProductData
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=0)
INFO:httpx:HTTP Request: POST http://local

Schema reset complete.
Is schema valid after reset? False

Schema Info:
Weaviate Schema Information:

Class: Product
  Object Count: 0
  Properties:
    - name (Type: text) - The name of the product.
    - ids (Type: text) - IDs of the products
    - manufacturer (Type: text) - The manufacturer of the product.
    - form_factor (Type: text) - The form of the product.
    - processor (Type: text) - The processor of the product.
    - core_count (Type: text) - The core count of the product.
    - processor_tdp (Type: text) - The processor TDP of the product.
    - memory (Type: text) - The memory of the product.
    - io (Type: text) - The IO of the product.
    - operating_system (Type: text) - The operating system of the product.
    - environmentals (Type: text) - The environmentals of the product.
    - certifications (Type: text) - The certifications of the product.
    - short_summary (Type: text) - The short summary of the product.
    - full_summary (Type: text) - The full produc

## 4. Product Service: CRUD Operations

We demonstrate Create, Read, Update, and Delete operations using the `ProductService`.

In [4]:
async def test_product_operations():
    print("\nTesting Product Service CRUD operations...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Create a product
    product_data = {
        "name": "ZumLink Series OEM ZPC ZPCSR",
        "manufacturer": "FreeWave Technologies",
        "form_factor": "ZPC: 63.5 x 38.1 x 10.2 mm, ZPCSR: 76.2 x 50.8 x 10.2 mm",
        "processor": "ARM Cortex-A8 1 GHz",
        "core_count": "1",
        "processor_tdp": "Not available",
        "memory": "RAM: 512 MB, Storage: 4 GB",
        "io": "Dual Row 20-pin header, Ethernet, Power, Serial, USB (Micro USB), RF Connector (MMCX)",
        "operating_system": "Debian-based Linux",
        "environmentals": "Operating Temperature: -40°C to 75°C (-40°F to 167°F), Humidity: 95% non-condensing",
        "certifications": "Class I Division 2 certified, UL, RoHS Directive 2011/65/EU",
        "short_summary": "The ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio designed for secure, long-range, low-power wireless communications in rugged environments.",
    }
    product_id = await product_service.create(product_data)
    print(f"Created product with ID: {product_id}")

    # Retrieve the product
    product = await product_service.get(product_id)
    print("\nRetrieved Product:")
    print(product)

    # Update the product
    update_data = {
        "short_summary": "Updated: The ZumLink Series OEM ZPC ZPCSR is a rugged industrial Ethernet radio for secure wireless communications.",
        "memory": "RAM: 1 GB, Storage: 8 GB",
    }
    await product_service.update(product_id, update_data)
    print("\nProduct updated.")

    updated_product = await product_service.get(product_id)
    print("\nUpdated Product:")
    print(updated_product)

    # Delete the product
    await product_service.delete(product_id)
    print("\nProduct deleted.")

    # Attempt to retrieve the deleted product
    deleted_product = await product_service.get(product_id)
    print("\nAttempt to retrieve deleted product:")
    print(deleted_product)  # Should be None

    await client.close()


await test_product_operations()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Product Service CRUD operations...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 61a4afbc-bbc6-48dc-9aef-70a8649f82bd


Created product with ID: 61a4afbc-bbc6-48dc-9aef-70a8649f82bd

Retrieved Product:
{'certifications': 'Class I Division 2 certified, UL, RoHS Directive 2011/65/EU', 'processor_tdp': 'Not available', 'full_product_description': None, 'io': 'Dual Row 20-pin header, Ethernet, Power, Serial, USB (Micro USB), RF Connector (MMCX)', 'ids': None, 'name': 'ZumLink Series OEM ZPC ZPCSR', 'form_factor': 'ZPC: 63.5 x 38.1 x 10.2 mm, ZPCSR: 76.2 x 50.8 x 10.2 mm', 'full_summary': None, 'environmentals': 'Operating Temperature: -40°C to 75°C (-40°F to 167°F), Humidity: 95% non-condensing', 'processor': 'ARM Cortex-A8 1 GHz', 'memory': 'RAM: 512 MB, Storage: 4 GB', 'manufacturer': 'FreeWave Technologies', 'operating_system': 'Debian-based Linux', 'core_count': '1', 'short_summary': 'The ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio designed for secure, long-range, low-power wireless communications in rugged environments.'}


INFO:httpx:HTTP Request: PATCH http://localhost:8080/v1/objects/Product/61a4afbc-bbc6-48dc-9aef-70a8649f82bd "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/objects/Product/61a4afbc-bbc6-48dc-9aef-70a8649f82bd "HTTP/1.1 204 No Content"



Product updated.

Updated Product:
{'certifications': 'Class I Division 2 certified, UL, RoHS Directive 2011/65/EU', 'processor_tdp': 'Not available', 'full_product_description': None, 'io': 'Dual Row 20-pin header, Ethernet, Power, Serial, USB (Micro USB), RF Connector (MMCX)', 'ids': None, 'name': 'ZumLink Series OEM ZPC ZPCSR', 'form_factor': 'ZPC: 63.5 x 38.1 x 10.2 mm, ZPCSR: 76.2 x 50.8 x 10.2 mm', 'full_summary': None, 'environmentals': 'Operating Temperature: -40°C to 75°C (-40°F to 167°F), Humidity: 95% non-condensing', 'processor': 'ARM Cortex-A8 1 GHz', 'memory': 'RAM: 1 GB, Storage: 8 GB', 'manufacturer': 'FreeWave Technologies', 'operating_system': 'Debian-based Linux', 'short_summary': 'Updated: The ZumLink Series OEM ZPC ZPCSR is a rugged industrial Ethernet radio for secure wireless communications.', 'core_count': '1'}

Product deleted.

Attempt to retrieve deleted product:
None


## 5. Product Service: Search Functionality

We test the search functionality by adding sample products that match the structure of actual products and performing semantic searches.

In [5]:
from weaviate.classes.query import Filter

In [6]:
async def test_product_search():
    print("\nTesting Product Service search functionality...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Create sample products for search testing
    products = [
        {
            "name": "ZumLink Series OEM ZPC ZPCSR",
            "manufacturer": "FreeWave Technologies",
            "processor": "ARM Cortex-A8 1 GHz",
            "memory": "RAM: 512 MB, Storage: 4 GB",
            "operating_system": "Debian-based Linux",
            "short_summary": "An industrial Ethernet radio for secure, long-range, low-power wireless communications in rugged environments.",
        },
        {
            "name": "Zoom AM Experimenter Kit",
            "manufacturer": "Logic Product Development",
            "processor": "Texas Instruments AM Processor",
            "memory": "Not available",
            "operating_system": "Linux",
            "short_summary": "A development kit for evaluating the functionality of the Texas Instruments AM processor.",
        },
        {
            "name": "LPC Development Kit",
            "manufacturer": "NXP Semiconductors",
            "processor": "NXP LPC",
            "memory": "32 MB SDRAM, 64 MB NAND Flash",
            "operating_system": "Embedded Linux",
            "short_summary": "A stable platform for building powerful user applications within an ARM Linux environment.",
        },
        {
            "name": "XMC Microcontroller",
            "manufacturer": "Infineon Technologies",
            "processor": "ARM Cortex-M",
            "memory": "Flash Memory",
            "operating_system": "Not available",
            "short_summary": "XMC Microcontroller with ARM Cortex-M processor and fixed flash wait states.",
        },
    ]
    for prod in products:
        await product_service.create(prod)

    # Perform a semantic search
    search_results = await product_service.search(query_text="industrial wireless communication device", limit=2)
    print(f"\n\nResults: {search_results}\n\n")
    print("\nSemantic Search Results:")
    for result in search_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    # Perform a filtered search
    filter_condition = Filter.by_property("manufacturer").equal("NXP Semiconductors")
    filtered_results = await product_service.search(query_text="development kit", filters=filter_condition, limit=3)
    print("\nFiltered Search Results (manufacturer: NXP Semiconductors):")
    for result in filtered_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    await client.close()


await test_product_search()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Product Service search functionality...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: e590ae6c-7da2-43d0-8891-6dcff12071e1
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: da76560e-470e-4fe2-b04f-ca595bfbf699
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 4c267e61-7819-4504-9db3-4059e0910862
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 53ed235b-050f-4e0e-a665-0b5541e1955d




Results: [{'certifications': None, 'processor_tdp': None, 'full_product_description': None, 'io': None, 'ids': None, 'full_summary': None, 'form_factor': None, 'name': 'ZumLink Series OEM ZPC ZPCSR', 'environmentals': None, 'processor': 'ARM Cortex-A8 1 GHz', 'memory': 'RAM: 512 MB, Storage: 4 GB', 'manufacturer': 'FreeWave Technologies', 'operating_system': 'Debian-based Linux', 'core_count': None, 'short_summary': 'An industrial Ethernet radio for secure, long-range, low-power wireless communications in rugged environments.', 'distance': 0.5396135449409485, 'certainty': 0.7301932573318481}, {'certifications': None, 'processor_tdp': None, 'full_product_description': None, 'io': None, 'ids': None, 'name': 'LPC Development Kit', 'form_factor': None, 'environmentals': None, 'full_summary': None, 'processor': 'NXP LPC', 'memory': '32 MB SDRAM, 64 MB NAND Flash', 'manufacturer': 'NXP Semiconductors', 'operating_system': 'Embedded Linux', 'core_count': None, 'short_summary': 'A stable pla

## 6. Product Service: Hybrid Search

We demonstrate the hybrid search functionality, which combines semantic and keyword search, using the updated product data structure.

In [7]:
async def test_hybrid_search():
    print("\nTesting Product Service hybrid search...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Perform a hybrid search
    hybrid_results = await product_service.hybrid_search(
        query_text="ARM processor development kit",
        limit=3,
        alpha=0.7,  # Adjust the balance between semantic and keyword search
    )
    print("\nHybrid Search Results:")
    for result in hybrid_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Processor: {result['processor']}")
        print(f"Operating System: {result['operating_system']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    await client.close()


await test_hybrid_search()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Product Service hybrid search...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Hybrid Search Results:
Name: ZumLink Series OEM ZPC ZPCSR
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A8 1 GHz
Operating System: Debian-based Linux
Summary: An industrial Ethernet radio for secure, long-range, low-power wireless communications in rugged environments.
---
Name: Zoom AM Experimenter Kit
Manufacturer: Logic Product Development
Processor: Texas Instruments AM Processor
Operating System: Linux
Summary: A development kit for evaluating the functionality of the Texas Instruments AM processor.
---
Name: LPC Development Kit
Manufacturer: NXP Semiconductors
Processor: NXP LPC
Operating System: Embedded Linux
Summary: A stable platform for building powerful user applications within an ARM Linux environment.
---


## 6. Product Service: Hybrid Search

We demonstrate the hybrid search functionality, which combines semantic and keyword search, using the updated product data structure.

In [ ]:
async def test_vector_search():
    print("\nTesting Product Service vector search...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Perform a vector search
    print("\nPerforming vector search for 'industrial IoT gateway'")
    vector_results = await product_service.vector_search(
        query="industrial IoT gateway",
        limit=3,
    )
    print("Vector Search Results:")
    for result in vector_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Processor: {result['processor']}")
        print(f"Operating System: {result['operating_system']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    await client.close()


await test_vector_search()

## 7. Route Service: Vector Search

We demonstrate the vector search functionality, which searches for products based on vector similarity.

In [2]:
async def test_vector_search():
    print("\nTesting Product Service vector search...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Perform a vector search
    print("\nPerforming vector search for 'industrial IoT gateway'")
    vector_results = await product_service.vector_search(
        query="industrial IoT gateway",
        limit=3,
    )
    print("Vector Search Results:")
    for result in vector_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Processor: {result['processor']}")
        print(f"Operating System: {result['operating_system']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    await client.close()


await test_vector_search()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Product Service vector search...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Performing vector search for 'industrial IoT gateway'
Vector Search Results:
Name: Industrial IoT Gateway
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A7
Operating System: Debian-based Linux
Summary: Rugged IoT gateway for industrial applications with advanced wireless capabilities.
---
Name: Industrial IoT Gateway
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A7
Operating System: Debian-based Linux
Summary: Rugged IoT gateway for industrial applications with advanced wireless capabilities.
---
Name: ZumLink Series OEM ZPC ZPCSR
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A8 1 GHz
Operating System: Debian-based Linux
Summary: An industrial Ethernet radio for secure, long-range, low-power wireless communications in rugged environments.
---


## 8. Product Service: Keyword Search

We demonstrate the keyword search functionality, which searches for products based on specific keywords.

In [3]:
async def test_keyword_search():
    print("\nTesting Product Service keyword search...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Perform a keyword search
    print("\nPerforming keyword search for 'ARM Cortex'")
    keyword_results = await product_service.keyword_search(
        query_text="ARM Cortex",
        properties_to_search_on=["processor", "short_summary"],
        limit=3,
    )
    print("Keyword Search Results:")
    for result in keyword_results:
        print(f"Name: {result['name']}")
        print(f"Manufacturer: {result['manufacturer']}")
        print(f"Processor: {result['processor']}")
        print(f"Summary: {result['short_summary']}")
        print("---")

    await client.close()


await test_keyword_search()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Product Service keyword search...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"



Performing keyword search for 'ARM Cortex'
Keyword Search Results:
Name: Industrial IoT Gateway
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A7
Summary: Rugged IoT gateway for industrial applications with advanced wireless capabilities.
---
Name: Industrial IoT Gateway
Manufacturer: FreeWave Technologies
Processor: ARM Cortex-A7
Summary: Rugged IoT gateway for industrial applications with advanced wireless capabilities.
---
Name: Custom ARM Dev Board
Manufacturer: NXP Semiconductors
Processor: ARM Cortex-A53
Summary: High-performance ARM development board for embedded applications.
---


## 9. Route Service: Testing and Demonstration

We test the `RouteService` by creating sample routes and demonstrating how to determine routing based on query text.

In [9]:
async def test_route_service():
    print("\nTesting Route Service...")
    await client.connect()
    route_service = weaviate_interface.get_service("Route")

    # Create sample routes
    routes = [
        {"route": "product", "description": "Queries about products, hardware, and specifications."},
        {"route": "support", "description": "Questions about customer support, troubleshooting, and services."},
        {"route": "sales", "description": "Inquiries about pricing, discounts, and purchasing."},
        {"route": "general", "description": "General inquiries, company information, and miscellaneous questions."},
    ]
    for route in routes:
        await route_service.create(route)
    print("Sample routes created.")

    # Test route search with sample queries
    test_queries = [
        "What are the specs of your latest gaming computer?",
        "I need help with my order, it hasn't arrived yet.",
        "Are there any discounts available for bulk purchases?",
        "What are your company's operating hours?",
        "Can you recommend a good laptop for video editing?",
    ]

    for query in test_queries:
        results = await route_service.search(query_text=query, limit=1)
        if results:
            route_name = results[0]["route"]
            print(f"\nQuery: '{query}'")
            print(f"Routed to: '{route_name}'")
        else:
            print(f"\nQuery: '{query}'")
            print("No route found.")
        print("-" * 30)

    await client.close()


await test_route_service()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing Route Service...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 68eb3eed-c704-4aa4-b664-a3852c1d6c0a
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: daf550a1-801e-4ae1-8373-12f05c39ea35
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: f6efd4a7-0f99-4df3-8a63-d75831f39d3f
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 8ed20cf1-272d-483f-8960-bae0f43b6cfe


Sample routes created.

Query: 'What are the specs of your latest gaming computer?'
Routed to: 'product'
------------------------------

Query: 'I need help with my order, it hasn't arrived yet.'
Routed to: 'support'
------------------------------

Query: 'Are there any discounts available for bulk purchases?'
Routed to: 'sales'
------------------------------

Query: 'What are your company's operating hours?'
Routed to: 'general'
------------------------------

Query: 'Can you recommend a good laptop for video editing?'
Routed to: 'product'
------------------------------


## 10. Batch Operations

We will test batch creation, retrieval, and deletion of products using the `ProductService`.


In [2]:
async def test_batch_operations():
    print("\nTesting batch operations...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Batch create products
    batch_products = [
        {"name": "Batch Product 1", "short_summary": "First batch product"},
        {"name": "Batch Product 2", "short_summary": "Second batch product"},
        {"name": "Batch Product 3", "short_summary": "Third batch product"},
    ]
    batch_ids = await product_service.batch_create_objects(batch_products)
    print(f"\nBatch created product IDs: {batch_ids}")

    # Batch retrieve products
    batch_retrieved = await product_service.get_all(limit=100)
    print("\nBatch retrieved products:")
    for product in batch_retrieved:
        print(product)

    # Batch delete products
    await product_service.batch_delete_objects(batch_ids)
    print("\nBatch deletion complete.")

    # Verify deletion
    remaining = await product_service.get_all(limit=100)
    print(f"\nRemaining products after batch deletion: {len(remaining)}")

    await client.close()

# Run the batch operations test function
await test_batch_operations()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing batch operations...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Inserting 3 objects into collection Product
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: a4e91886-d669-4e8c-94d1-99c449aba516
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 21ec1669-bd09-4235-aa9e-8fb31a837599
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 928bdad7-00ae-4cce-ad65-dd5079082e47
INFO:weaviate_interface.weaviate_client:Batch insert completed. 3 objects inserted.
ERROR:weaviate_interface.base_service:Error retrieving all objects from Product: '_QueryCollectionAsync' object has no attribute 'get_all'
INFO:weaviate_interface.weaviate_client:Batch de


Batch created product IDs: [UUID('928bdad7-00ae-4cce-ad65-dd5079082e47'), UUID('a4e91886-d669-4e8c-94d1-99c449aba516'), UUID('21ec1669-bd09-4235-aa9e-8fb31a837599')]

Batch retrieved products:

Batch deletion complete.

Remaining products after batch deletion: 0


## 11. Aggregation Queries

We demonstrate aggregation queries using the `ProductService` to group products and calculate aggregate properties based on the updated product data structure. Aggregation is useful for generating summary statistics and insights from your data.

In [5]:
async def test_aggregation():
    print("\nTesting aggregation queries...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Create additional products for aggregation
    products = [
        {
            "name": "Custom ARM Dev Board",
            "manufacturer": "NXP Semiconductors",
            "processor": "ARM Cortex-A53",
            "memory": "2 GB RAM, 16 GB eMMC",
            "operating_system": "Embedded Linux",
            "short_summary": "High-performance ARM development board for embedded applications.",
        },
        {
            "name": "Industrial IoT Gateway",
            "manufacturer": "FreeWave Technologies",
            "processor": "ARM Cortex-A7",
            "memory": "1 GB RAM, 8 GB Flash",
            "operating_system": "Debian-based Linux",
            "short_summary": "Rugged IoT gateway for industrial applications with advanced wireless capabilities.",
        },
    ]
    for prod in products:
        await product_service.create(prod)

    # Perform aggregation
    agg_results = await product_service.aggregate(
        group_by=["manufacturer", "operating_system"], properties=["processor"]
    )
    print("\nAggregation Results:")
    for group in agg_results.get("groups", []):
        print(f"Manufacturer: {group['groupBy'][0]}, Operating System: {group['groupBy'][1]}")
        print(f"  Count: {group['count']}")
        print(f"  Processors:")
        for prop in group.get("properties", []):
            if prop["name"] == "processor":
                print(f"    Unique Values: {prop.get('uniqueCount', 0)}")
        print("-" * 30)

    await client.close()


await test_aggregation()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing aggregation queries...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: b0e2bf89-037a-4413-a48b-d3a294987af5
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: ac441ddf-accb-4a33-b6dc-b6fa45214802
ERROR:weaviate_interface.weaviate_client:Error performing aggregation in collection Product: '_AggregateCollectionAsync' object has no attribute 'group_by'



Aggregation Results:


In [6]:
async def test_aggregation():
    print("\nTesting aggregation queries...")
    await client.connect()
    product_service = weaviate_interface.get_service("Product")

    # Create additional products for aggregation
    products = [
        {
            "name": "Custom ARM Dev Board",
            "manufacturer": "NXP Semiconductors",
            "processor": "ARM Cortex-A53",
            "memory": "2 GB RAM, 16 GB eMMC",
            "operating_system": "Embedded Linux",
            "short_summary": "High-performance ARM development board for embedded applications.",
        },
        {
            "name": "Industrial IoT Gateway",
            "manufacturer": "FreeWave Technologies",
            "processor": "ARM Cortex-A7",
            "memory": "1 GB RAM, 8 GB Flash",
            "operating_system": "Debian-based Linux",
            "short_summary": "Rugged IoT gateway for industrial applications with advanced wireless capabilities.",
        },
    ]
    for prod in products:
        await product_service.create(prod)

    # Perform aggregation
    print("\nPerforming aggregation on products")
    agg_results = await product_service.aggregate()
    print("\nAggregation Results:")
    print(f"Total number of products: {agg_results.total_count}")

    # Demonstrate a workaround using get_all and manual grouping
    all_products = await product_service.get_all(limit=100)

    # Manual grouping by manufacturer
    manufacturers = {}
    for product in all_products:
        manufacturer = product.get("manufacturer", "Unknown")
        if manufacturer not in manufacturers:
            manufacturers[manufacturer] = []
        manufacturers[manufacturer].append(product)

    print("\nManual grouping by manufacturer:")
    for manufacturer, products in manufacturers.items():
        print(f"Manufacturer: {manufacturer}")
        print(f"  Number of products: {len(products)}")
        print(f"  Products:")
        for product in products:
            print(f"    - {product['name']}")
        print("-" * 30)

    await client.close()


await test_aggregation()


Testing aggregation queries...


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: ac6bf07b-35bf-49fd-ac8f-cb6f35ded85d
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 20d70128-3fba-4f2d-8f2d-f827886084e4
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/graphql "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Aggregation results: AggregateReturn(properties={}, total_count=18)



Performing aggregation on products

Aggregation Results:
Total number of products: 18

Manual grouping by manufacturer:
Manufacturer: FreeWave Technologies
  Number of products: 7
  Products:
    - Industrial IoT Gateway
    - Industrial IoT Gateway
    - Industrial IoT Gateway
    - Industrial IoT Gateway
    - ZumLink Series OEM ZPC ZPCSR
    - Industrial IoT Gateway
    - ZumLink Series OEM ZPC ZPCSR
------------------------------
Manufacturer: Infineon Technologies
  Number of products: 2
  Products:
    - XMC Microcontroller
    - XMC Microcontroller
------------------------------
Manufacturer: NXP Semiconductors
  Number of products: 7
  Products:
    - Custom ARM Dev Board
    - LPC Development Kit
    - LPC Development Kit
    - Custom ARM Dev Board
    - Custom ARM Dev Board
    - Custom ARM Dev Board
    - Custom ARM Dev Board
------------------------------
Manufacturer: Logic Product Development
  Number of products: 2
  Products:
    - Zoom AM Experimenter Kit
    - Zoom A

## 12. RawProductDataService Testing


In [2]:
async def test_raw_product_data_service():
    print("\nTesting RawProductDataService...")
    await client.connect()
    raw_product_data_service = weaviate_interface.raw_product_data_service

    # Create a sample raw product data
    raw_data = {
        "product_id": "test_product_001",
        "raw_data": "This is a test product with various specifications and details..."
    }
    raw_data_id = await raw_product_data_service.create(raw_data)
    print(f"Created raw product data with ID: {raw_data_id}")

    # Retrieve the raw product data
    retrieved_data = await raw_product_data_service.get_by_product_id("test_product_001")
    print("\nRetrieved Raw Product Data:")
    print(retrieved_data)

    # Update the raw product data
    update_data = {
        "raw_data": "Updated test product data with more information..."
    }
    await raw_product_data_service.update(raw_data_id, update_data)
    print("\nRaw Product Data updated.")

    updated_data = await raw_product_data_service.get_by_product_id("test_product_001")
    print("\nUpdated Raw Product Data:")
    print(updated_data)

    # Delete the raw product data
    await raw_product_data_service.delete_by_product_id("test_product_001")
    print("\nRaw Product Data deleted.")

    # Attempt to retrieve the deleted data
    deleted_data = await raw_product_data_service.get_by_product_id("test_product_001")
    print("\nAttempt to retrieve deleted raw product data:")
    print(deleted_data)  # Should be None

    await client.close()

await test_raw_product_data_service()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing RawProductDataService...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: f56f0017-f462-4521-85f3-6d45fee9ddb8


Created raw product data with ID: f56f0017-f462-4521-85f3-6d45fee9ddb8

Retrieved Raw Product Data:
{'raw_data': 'This is a test product with various specifications and details...', 'product_id': 'test_product_001'}


INFO:httpx:HTTP Request: PATCH http://localhost:8080/v1/objects/RawProductData/f56f0017-f462-4521-85f3-6d45fee9ddb8 "HTTP/1.1 204 No Content"



Raw Product Data updated.

Updated Raw Product Data:
{'raw_data': 'This is a test product with various specifications and details...', 'product_id': 'test_product_001'}

Raw Product Data deleted.

Attempt to retrieve deleted raw product data:
None


## 13. ProductSearchResultService Testing


In [2]:
async def test_product_search_result_service():
    print("\nTesting ProductSearchResultService...")
    await client.connect()
    product_search_result_service = weaviate_interface.get_service("ProductSearchResult")

    # Create sample search results
    search_results = [
        {
            "product_id": "test_product_001",
            "search_query": "embedded system with ARM processor",
            "search_result": "Found 3 matching products...",
            "data_source": "https://example.com/search"
        },
        {
            "product_id": "test_product_001",
            "search_query": "industrial IoT gateway",
            "search_result": "Found 2 matching products...",
            "data_source": "https://example.com/search"
        }
    ]

    for result in search_results:
        result_id = await product_search_result_service.create(result)
        print(f"Created search result with ID: {result_id}")

    # Retrieve search results for a product
    retrieved_results = await product_search_result_service.get_by_product_id("test_product_001")
    print("\nRetrieved Search Results:")
    for result in retrieved_results:
        print(result)

    # Delete search results for a product
    await product_search_result_service.delete_by_product_id("test_product_001")
    print("\nSearch results deleted.")

    # Verify deletion
    remaining_results = await product_search_result_service.get_by_product_id("test_product_001")
    print("\nRemaining search results after deletion:")
    print(remaining_results)  # Should be an empty list

    await client.close()

await test_product_search_result_service()


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing ProductSearchResultService...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 70914228-9086-453b-877a-038fc6831c58


Created search result with ID: 70914228-9086-453b-877a-038fc6831c58


INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: a7c0037a-bef3-4b35-be8a-2e35bac35a3f


Created search result with ID: a7c0037a-bef3-4b35-be8a-2e35bac35a3f

Retrieved Search Results:
{'data_source': 'https://example.com/search', 'search_result': 'Found 3 matching products...', 'search_query': 'embedded system with ARM processor', 'product_id': 'test_product_001'}
{'data_source': 'https://example.com/search', 'search_result': 'Found 2 matching products...', 'search_query': 'industrial IoT gateway', 'product_id': 'test_product_001'}
{'data_source': 'https://example.com/search', 'search_result': 'Found 3 matching products...', 'search_query': 'embedded system with ARM processor', 'product_id': 'test_product_001'}
{'data_source': 'https://example.com/search', 'search_result': 'Found 2 matching products...', 'search_query': 'industrial IoT gateway', 'product_id': 'test_product_001'}

Search results deleted.

Remaining search results after deletion:
[]


## 14. ProductDataChunkService Testing


In [2]:
async def test_product_data_chunk_service():
    print("\nTesting ProductDataChunkService...")
    await client.connect()
    product_data_chunk_service = weaviate_interface.get_service("ProductDataChunk")

    # Create sample data chunks
    chunks = [
        "This is the first chunk of product data...",
        "This is the second chunk with more details...",
        "And this is the third chunk with specifications..."
    ]

    chunk_ids = await product_data_chunk_service.create_chunks(
        chunks=chunks,
        product_id="test_product_001",
        source_type="raw_data",
        source_id="raw_data_001"
    )
    print(f"Created {len(chunk_ids)} data chunks")

    # Retrieve chunks for a product
    retrieved_chunks = await product_data_chunk_service.get_by_product_id("test_product_001")
    print("\nRetrieved Data Chunks:")
    for chunk in retrieved_chunks:
        print(chunk)

    # Perform a semantic search on chunks
    search_results = await product_data_chunk_service.semantic_search(
        query="product specifications",
        product_id="test_product_001",
        limit=2
    )
    print("\nSemantic Search Results:")
    for result in search_results:
        print(result)

    # Delete chunks for a product
    await product_data_chunk_service.delete_by_product_id("test_product_001")
    print("\nData chunks deleted.")

    # Verify deletion
    remaining_chunks = await product_data_chunk_service.get_by_product_id("test_product_001")
    print("\nRemaining data chunks after deletion:")
    print(remaining_chunks)  # Should be an empty list

    await client.close()

await test_product_data_chunk_service()

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"



Testing ProductDataChunkService...


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Inserting 3 objects into collection ProductDataChunk
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 53d3b1bb-bb41-4d3c-a3d3-d65867dcf3fa
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/objects "HTTP/1.1 200 OK"
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 2e5a8323-8532-4663-9370-e2e6fbae0e3a
INFO:weaviate_interface.weaviate_client:Object inserted with UUID: 1eb099f7-37d0-479c-8056-1f62ca61e4f9
INFO:weaviate_interface.weaviate_client:Batch insert completed. 3 objects inserted.


Created 3 data chunks

Retrieved Data Chunks:
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'This is the first chunk of product data...'}
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'And this is the third chunk with specifications...'}
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'This is the second chunk with more details...'}
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'This is the second chunk with more details...'}
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'This is the first chunk of product data...'}
{'source_type': 'raw_data', 'source_id': 'raw_data_001', 'product_id': 'test_product_001', 'chunk_text': 'And this is the third chunk with specifications...'}

Semantic Search Results:


## Conclusion

This notebook has tested various functionalities of the `weaviate_interface` subpackage, ensuring that it works as expected with the latest `WeaviateAsyncClient` API. We've demonstrated:

1. Connection and schema management
2. CRUD operations for products
3. Various search functionalities including semantic, filtered, hybrid, vector, and keyword searches
4. Route service for query routing
5. Batch operations for efficient data manipulation
6. Aggregation queries for data analysis

These tests cover a wide range of use cases and should help ensure the reliability and functionality of the `weaviate_interface` subpackage.
